This WAS an example to fetch KEGG pathways.  
Here we convert it to Wikipathways access.

Requirments
In addition to basic cyREST setup, you need to install the following Cytoscpae App to run this workflow:
Wikipathways app
Input and Output
Input - Disease name
Output - Cytoscape session file containing all KEGG pathways known to be related to the disease.
 containing all KEGG pathways known to be related to the disease.


In [50]:
import requests
import json
import pandas as pd
import io
from IPython.display import Image

# Basic Setup
PORT_NUMBER = 1234
BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'
WP_API_URL = 'http://webservice.wikipathways.org/'

# Header for posting data to the server as JSON
HEADERS = {'Content-Type: application/json'}

# Delete all networks in current session
requests.delete(BASE + 'session')


<Response [200]>

In [59]:
import xmltodict
#import untangle

# Find information about cancer from Wikipathways.
query = 'cancer'

res = requests.get(WP_API_URL + 'findPathwaysByText?query=' + query)
pathway_list = res.content.decode('utf8')
doc = xmltodict.parse(pathway_list)

response = doc['ns1:findPathwaysByTextResponse']
print ('RESPONSE: ' + str(response.keys()))

result = response['ns1:result']

print ('RESULT: ' + str(result['ns2:score']))

#for line in result.keys():
#   print (line)


#print (doc[ns1::result][ns2:score])
#disease_df = pd.read_csv(io.StringIO(pathway_list), delimiter='\t',  header=None, names=['id', 'name'])
#disease_df

RESPONSE: odict_keys(['@xmlns:ns1', '@xmlns:ns2', 'ns1:result'])


TypeError: list indices must be integers or slices, not str

## Piping the result to KEGG Pathway database to get list of related pathways

In [7]:
disease_ids = disease_df['id']
disease_urls = disease_ids.apply(lambda x: KEGG_API_URL + 'get/' + x)

def disease_parser(entry):
    lines = entry.split('\n')
    data = {}
    
    last_key = None
    for line in lines:
        if '///' in line:
            return data
        
        parts = line.split(' ')
        if parts[0] is not None and len(parts[0]) != 0:
            last_key = parts[0]
            data[parts[0]] = line.replace(parts[0], '').strip()
        else:
            last_val = data[last_key]
            data[last_key] = last_val + '|' + line.strip()
    return data   

result = []
for url in disease_urls:
        res = requests.get(url)
        rows = disease_parser(res.content.decode('utf8'))
        result.append(rows)

In [9]:
disease_df = pd.DataFrame(result)
pathways = disease_df['PATHWAY'].dropna().unique()

p_urls = []
for pathway in pathways:
    entries = pathway.split('|')
    for en in entries:
        url = KEGG_API_URL + 'get/' + en.split(' ')[0].split('(')[0] + '/kgml'
        p_urls.append(url)

In [8]:
def create_from_list(network_list):
    server_res = requests.post(BASE + 'networks?source=url&collection=' + query, data=json.dumps(network_list), headers=HEADERS)
    return server_res.json()

url_list = list(set(p_urls))
pathway_suids = create_from_list(url_list)

NameError: name 'p_urls' is not defined